In [1]:
import csv
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv(r"answer_sql3.csv", sep='|')
len(data)

3206

In [3]:
relax_rate = 1.05
contract_rate = 0.95
relax_rate_change = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6]
contract_rate_change = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [4]:
len(data[(data["l_shipmode"] == "TRUCK") & (data["l_shipinstruct"] == "DELIVER IN PERSON")])

108

In [5]:
len(data[(data["l_linenumber"] == 6) & (data["l_shipmode"] == "SHIP")])

29

## relax

In [6]:
data2 = data[(data['c_nationkey'] == 23)]
print(len(data2))
print(len(data2)*relax_rate)

122
128.1


In [7]:
data3 = data[(data['l_shipmode'] == "MAIL")]
print(len(data3))
print(len(data3)*relax_rate)

501
526.0500000000001


In [8]:
for r in relax_rate_change:
    print(len(data3) * r)

551.1
601.1999999999999
651.3000000000001
701.4
751.5
801.6


## contract

In [9]:
data3 = data[(data['c_nationkey'] == 2)]
print(len(data3))
print(len(data3) * contract_rate)

102
96.89999999999999


In [10]:
data1 = data[(data['o_orderstatus'] == "P") & (data['l_returnflag'] == "N")]
print(len(data1))
print(len(data1) * contract_rate)

386
366.7


In [11]:
data3 = data[(data['l_shipmode'] == "SHIP")]
print(len(data3))
print(len(data3) * contract_rate)

458
435.09999999999997


In [12]:
data3 = data[(data['l_shipinstruct'] == "COLLECT COD")]
print(len(data3))
print(len(data3) * contract_rate)

839
797.05


In [13]:
for r in contract_rate_change:
    print(len(data3) * r)


335.6
419.5
503.4
587.3
671.2
755.1


In [14]:
data3 = data[(data['l_linenumber'] == 5)]
print(len(data3))
print(len(data3) * contract_rate)

341
323.95


## refinement 1

In [15]:
data1 = data[(data['c_nationkey'] == 12)]
print(len(data1))
print(len(data1) * 1.05)

126
132.3


In [17]:
for s in [1.02, 1.04, 1.06, 1.08, 1.10, 1.12]:
    print(len(data1) * s)

128.52
131.04
133.56
136.08
138.60000000000002
141.12


In [18]:
data3 = data[(data['l_shipmode'] == "TRUCK")]
print(len(data3))
print(len(data3) * 0.95)

445
422.75


## refinement 2

In [81]:
data1 = data[(data["l_shipmode"] == "TRUCK") & (data["o_orderstatus"] == "P")]
print(len(data1))
print(len(data1) * 0.95)

123
116.85


In [82]:
data1 = data[(data["l_linenumber"] == 6) & (data["l_shipmode"] == "SHIP")]
print(len(data1))
print(len(data1) * 1.05)


29
30.450000000000003


In [83]:
for s in [1.02, 1.04, 1.06, 1.08, 1.10, 1.12]:
    print(len(data1) * s)

29.580000000000002
30.16
30.740000000000002
31.32
31.900000000000002
32.480000000000004


# refine3 using

In [97]:
data1 = data[(data["l_shipinstruct"] == "DELIVER IN PERSON") & (data["o_orderstatus"] == "P") & (data["o_orderpriority"] == "2-HIGH")]
print(len(data1))
print(len(data1) * 0.95)

42
39.9


In [98]:
for s in [0.88, 0.9, 0.92, 0.94, 0.96, 0.98]:
    print(len(data1) * s)

36.96
37.800000000000004
38.64
39.48
40.32
41.16


In [99]:
data1 = data[(data["l_linenumber"] == 6) & (data["l_shipmode"] == "SHIP") & (data["o_orderstatus"] == "F") & (data["o_orderpriority"] == "1-URGENT")]
print(len(data1))
print(len(data1) * 1.05)


3
3.1500000000000004


# find a good refine constraint

In [31]:
# relax: PVT_head: ['o_orderdate', 'l_shipdate', 'c_mktsegment__AUTOMOBILE', 'c_mktsegment__MACHINERY', 'c_mktsegment__HOUSEHOLD', 'c_mktsegment__FURNITURE']

## join result

In [16]:
customer = pd.read_csv("../../../../../InputData/TPC-H/100Mdata/customer.tbl", sep='|')
orders = pd.read_csv("../../../../../InputData/TPC-H/100Mdata/orders.tbl", sep='|')
joined = pd.merge(left=customer, right=orders, how="inner", left_on="c_custkey", right_on="o_custkey")


In [36]:
lineitem = pd.read_csv("../../../../../InputData/TPC-H/100Mdata/lineitem.tbl", sep='|')
joined = pd.merge(left=joined, right=lineitem, how="inner", left_on="o_orderkey", right_on="l_orderkey")

In [37]:
len(joined)

600572

In [38]:
# joined.to_csv("sql3_joined.csv", sep='|', index=False)

In [39]:
# data analysis

In [34]:
joined = pd.read_csv("sql3_joined.csv", sep='|')

In [35]:
print(len(joined))

600572


In [75]:
g1 = joined[
                (joined["l_shipinstruct"] == "DELIVER IN PERSON")\
                & (joined["o_orderpriority"] == "2-HIGH") &  (joined["o_orderstatus"] == "P")]
print(len(g1))

956


In [81]:
g2 = joined[(joined["l_linenumber"] == 6) & (joined["l_shipmode"] == "SHIP") & (joined["o_orderstatus"] == "F") & (joined["o_orderpriority"] == "2-HIGH")]
print(len(g2))


624


In [36]:
joined['o_orderstatus'].unique().tolist()


['O', 'F', 'P']

In [37]:
print(len(joined[(joined['l_shipmode'] == "TRUCK") & (joined['o_orderstatus'] == "P")]))

print(len(joined[(joined['l_shipmode'] == "SHIP") & (joined['l_linenumber'] == 6)]))



2630
6074


In [38]:
joined["o_orderstatus"].unique().tolist()

['O', 'F', 'P']

# refine1 using

In [92]:
# original query
data1 = joined[(joined['o_orderdate'] <  19950328)]
data1 = data1[(data1['l_shipdate'] >  19950328)]
data1 = data1[(data1['c_mktsegment'] == "BUILDING")]
print(len(data1))
# res1 = data1[(data1["l_shipmode"] == "TRUCK") \
#                 & (data1["l_shipinstruct"] == "DELIVER IN PERSON")\
#                 & (data1["o_orderpriority"] == "2-HIGH") & (joined["o_orderstatus"] == "P")]
res1 = data1[
                 (data1["l_shipinstruct"] == "DELIVER IN PERSON")\
                & (data1["o_orderpriority"] == "2-HIGH") & (joined["o_orderstatus"] == "P")]
print(len(res1))
print(len(res1) * 0.95)

res2 = data1[(data1["l_linenumber"] == 6) & (data1["l_shipmode"] == "SHIP") & (data1["o_orderstatus"] == "F")& (data1["o_orderpriority"] == "1-URGENT")]
print(len( res2))
print(len( res2) * 1.05)


3206
42
39.9
3
3.1500000000000004


/var/folders/sy/m1107g5j24x3w6mcyb9njvyw0000gn/T/ipykernel_77020/4101107641.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res1 = data1[


In [93]:

data1 = joined[(joined['o_orderdate'] <  19950328)]
data1 = data1[(data1['l_shipdate'] >  19950328)]
data1 = data1[(data1['c_mktsegment'] == "HOUSEHOLD")]

print(len(data1))

res1 = data1[
                (data1["l_shipinstruct"] == "DELIVER IN PERSON")\
                & (data1["o_orderpriority"] == "2-HIGH") & (joined["o_orderstatus"] == "P")]
print(len(res1))

res2 = data1[(data1["l_linenumber"] == 6) & (data1["l_shipmode"] == "SHIP") & (data1["o_orderstatus"] == "F")& (data1["o_orderpriority"] == "1-URGENT")]
print(len( res2))


2951
36
6


/var/folders/sy/m1107g5j24x3w6mcyb9njvyw0000gn/T/ipykernel_77020/313060801.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  res1 = data1[


In [44]:
data1 = joined[(joined['o_orderdate'] <  19950328)]
data1 = data1[(data1['l_shipdate'] >  19950328)]
data1 = data1[(data1['c_mktsegment'] == "AUTOMOBILE")]
len(data1)


3017

In [72]:
print(len(joined[(joined['l_shipmode'] == "TRUCK") & (joined['o_orderstatus'] == "P")]))

print(len(joined[(joined['l_shipmode'] == "SHIP") & (joined['l_linenumber'] == 6)]))



2630
6074


In [73]:
# original query
data1 = joined[(joined['o_orderdate'] <  19950328)]
data1 = data1[(data1['l_shipdate'] >  19950328)]
data1 = data1[(data1['c_mktsegment'] == "BUILDING")]
print(len(data1))
print(len(data1[(data1["l_shipmode"] == "TRUCK") & (data1["o_orderstatus"] == "P")]))
print(len(data1[(data1["l_shipmode"] == "TRUCK") & (data1["o_orderstatus"] == "P")]) * 0.95)
print(len(data1[(data1["l_linenumber"] == 6) & (data1["l_shipmode"] == "SHIP")]))
print(len(data1[(data1["l_linenumber"] == 6) & (data1["l_shipmode"] == "SHIP")]) * 1.05)

3206
123
116.85
29
30.450000000000003


In [74]:

data1 = joined[(joined['o_orderdate'] <  19950328)]
data1 = data1[(data1['l_shipdate'] >  19950328)]
data1 = data1[(data1['c_mktsegment'] == "HOUSEHOLD")]
print(len(data1))
print(len(data1[(data1["l_shipmode"] == "TRUCK") & (data1["o_orderstatus"] == "P")]))
print(len(data1[(data1["l_linenumber"] == 6) & (data1["l_shipmode"] == "SHIP")]))

2951
111
31


## date

In [36]:
s = data["o_orderdate"].values.tolist()
s.sort()
print(s[0])
print(s[-1])

1994-11-29
1995-03-27


In [37]:
s = data["l_shipdate"].values.tolist()
s.sort()
print(s[0])
print(s[-1])

1995-03-29
1995-07-26
